In [1]:
import numpy as np
import pandas as pd,os
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from scipy.stats import wilcoxon, friedmanchisquare, rankdata
import scikit_posthocs as sp
import pingouin as pg
import seaborn as sns
import matplotlib.pyplot as plt
from library.utils import read_data

In [13]:
clean = pd.read_csv("Clean.csv",header=[0,1,2],index_col=0)  # All_filter_MCC.csv
noisy = pd.read_csv("Noisy.csv",header=[0,1,2],index_col=0)  #
clean = clean.drop(columns=['pr_rec_score'],level=2).droplevel(2,axis=1)  # matthews_corrcoef
noisy = noisy.drop(columns=['pr_rec_score'],level=2).droplevel(2,axis=1)
df = clean-noisy
clean.shape, noisy.shape

((32, 30), (32, 30))

## Dataset Characteristic vs MCC

In [16]:
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]

In [19]:
df.columns

MultiIndex([( 'smote',  'dt'),
            ( 'smote',  'lr'),
            ( 'smote',  'nb'),
            ( 'smote', 'svm'),
            ( 'smote', 'knn'),
            ( 'smote',  'rf'),
            (   'rus',  'dt'),
            (   'rus',  'lr'),
            (   'rus',  'nb'),
            (   'rus', 'svm'),
            (   'rus', 'knn'),
            (   'rus',  'rf'),
            ('wilson',  'dt'),
            ('wilson',  'lr'),
            ('wilson',  'nb'),
            ('wilson', 'svm'),
            ('wilson', 'knn'),
            ('wilson',  'rf'),
            ( 'tomek',  'dt'),
            ( 'tomek',  'lr'),
            ( 'tomek',  'nb'),
            ( 'tomek', 'svm'),
            ( 'tomek', 'knn'),
            ( 'tomek',  'rf'),
            (  'None',  'dt'),
            (  'None',  'lr'),
            (  'None',  'nb'),
            (  'None', 'svm'),
            (  'None', 'knn'),
            (  'None',  'rf')],
           names=['imb', 'clf'])

In [23]:
info = pd.DataFrame(columns=['noise','FP','FN','TP','TN','nIR','IR','PR','Rec','MCC'])
for d in DATASETS:
    X,y_noisy,y_real = read_data(d,stats=False)
    noise = (y_noisy!=y_real).sum()/len(y_noisy)
    tn, fp, fn, tp = confusion_matrix(y_real,y_noisy).ravel()
    nIR = (y_noisy==0).sum()/y_noisy.sum()
    IR = (y_real==0).sum()/y_real.sum()
    pr = precision_score(y_real,y_noisy)
    rec = recall_score(y_real,y_noisy)
    for col in df.columns:
        info.loc[len(info)] = [noise,fp,fn,tp,tn,nIR,IR,pr,rec]+[df.loc[d,col]]
        
info['size'] = info['TN']+info['FN']+info['TP']+info['FP']
info.shape,info.isna().sum().sum()

((960, 11), 0)

In [ ]:
1-2/(12.648854+1)

In [ ]:
dataStat = df.loc[::dependent.shape[1]].drop(columns=['MCC'])
dataStat.corr()

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(df['noise'],df['MCC'],s=20,color='y')
f = sns.regplot(x='noise',y='MCC',data=df,robust=False,order=2,ci=95,scatter=False); #line_kws={'color':'green'}
plt.xlabel("Noise Level");

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(df['nIR'],df['MCC'],s=20,color='y')
f = sns.regplot(x='nIR',y='MCC',data=df,robust=False,order=2,ci=95,scatter=False); #line_kws={'color':'green'}
plt.xlabel("Noise Level");

In [ ]:
for col in df:
    if col=='MCC': continue
    print(col)
    print(pg.corr(df[col],df['MCC'],method='percbend'),'\n\n') #shepherd , percbend